In [1]:
import nltk; nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cbeams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
# Import Dataset
df = pd.read_json('../raw_data/all_data_df_stripped.json')
print(df.artist.unique())
df.head()

['Adam Devine' 'Adam Hess' 'Adam Hills' 'Adam Sandler' 'Adel Karam'
 'Aisling Bea' 'Al Madrigal' 'Al Murray' 'Al Porter' 'Alan Carr'
 'Alex Edelman' 'Ali Wong' 'Alistair McGowan' 'Amanda Seales'
 'Amy Schumer' 'Andi Osho' 'Andrew Lawrence' 'Andrew Maxwell'
 'Andy Parsons' 'Andy Woodhull' 'Angela Barnes' 'Anjelah Johnson'
 'Anthony Jeselnik' 'Ari Shaffir' 'Arj Barker' 'Arsenio Hall'
 'Aziz Ansari' 'Bert Kreischer' 'Big Jay Oakerson' 'Bill Burr'
 'Bill Hicks' 'Bill Maher' 'Billy Connolly' 'Bo Burnham' 'Bob Monkhouse'
 'Brad Williams' 'Brent Morin' 'Brian Regan' 'Bridget Everett'
 'Cedric The Entertainer' 'Celia Pacquola' 'Chelsea Peretti'
 'Chris Addison' "Chris D'Elia" 'Chris Gethard' 'Chris McCausland'
 'Chris Moyles' 'Chris Ramsey' 'Chris Rock' 'Chris Tucker'
 'Christina Pazsitzky' 'Colin Quinn' 'Craig Ferguson' 'Cristela Alonzo'
 'D.l. Hughley' 'Dan Soder' 'Dana Carvey' 'Dane Baptiste' 'Daniel Tosh'
 'Danny Bhoy' 'Dara Ó Briain' 'Darren Harriott' 'Dave Allen' 'Dave Attell'
 'Dave Cha

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then,full_transcript_clean
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0,man guy much seat guy jacked fully juiced man ...
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0,yes yes yes lovely apologise state voice went ...
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0,tonight london london live amazing act present...
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0,tonight hair lost bet british paralympic team ...
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0,start saying something probably never heard co...


In [4]:
# Convert to list
data = df.full_transcript_clean.values.tolist()

pprint(data[:1])

['man guy much seat guy jacked fully juiced man excited guy blast tonight '
 'excited excited though adult man excited lock littlekidlevel geeked remember '
 'kid open present vocabulary much mama however guy talked weirdo said thing '
 'mama much mama vocabulary went like face adult lock adultass thing kid '
 'school drop enjoy lunch today made ham cheese question quality meat made '
 'littlekidlevel excited face month child swear mom gave super bouncy ball '
 'birthday guy poor parent cool gave bouncy ball month straight driveway like '
 'snatched midair phrase use youth midair nothing midair adult child caught '
 'midair mom soaring abyss snatched midair midair mom midair midair midair mom '
 'jesus christ course caught midair honey anything catch air would technically '
 'midair caught ground called picking idiot dumb little boy kids– cool kid '
 'still getting level geeked still getting fully geeked elevator long ago big '
 'deal stair stupid stupid elevator crappy day adult weari

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['man', 'guy', 'much', 'seat', 'guy', 'jacked', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited', 'excited', 'though', 'adult', 'man', 'excited', 'lock', 'littlekidlevel', 'geeked', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'mama', 'however', 'guy', 'talked', 'weirdo', 'said', 'thing', 'mama', 'much', 'mama', 'vocabulary', 'went', 'like', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'question', 'quality', 'meat', 'made', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'cool', 'gave', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase', 'use', 'youth', 'midair', 'nothing', 'midair', 'adult', 'child', 'caught', 'midair', 'mom', 'soaring', 'abyss', 'snatched', 'midair', 'midair', 'mom', 'midair', 'midair', 'midair', 'mom', 'jesus', 'chris

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['man', 'guy', 'much', 'seat', 'guy', 'jacked', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited_excited', 'though', 'adult', 'man', 'excited', 'lock', 'littlekidlevel', 'geeked', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'mama', 'however', 'guy', 'talked', 'weirdo', 'said', 'thing', 'mama', 'much', 'mama', 'vocabulary', 'went', 'like', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid_school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'question', 'quality', 'meat', 'made', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'cool', 'gave', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase_use', 'youth', 'midair', 'nothing', 'midair', 'adult', 'child', 'caught', 'midair', 'mom', 'soaring', 'abyss', 'snatched', 'midair', 'midair', 'mom', 'midair', 'midair', 'midair', 'mom', 'jesus_christ', 'course',

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['guy', 'much', 'seat', 'guy', 'jack', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited_excite', 'adult', 'man', 'excite', 'lock', 'littlekidlevel', 'geeke', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'however', 'talk', 'say', 'thing', 'much', 'go', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'make', 'ham', 'cheese', 'question', 'quality', 'meat', 'make', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'give', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'give', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'snatch', 'youth', 'adult', 'child', 'catch', 'mom', 'soar', 'abyss', 'snatch', 'midair', 'mom', 'mom', 'course', 'catch', 'midair', 'honey', 'catch', 'air', 'technically', 'midair', 'catch', 'ground', 'call', 'pick', 'idiot', 'dumb', 'kid', 'cool', 'kid', 'still', 'get', 'level', 'geeke', 'still', 'get', 'fully', 'geeke', 'elevator', 'big_de

In [38]:
### additional words to remove from the scripts
words_to_remove = ['thank', 'cheering', 'recorded', 'applause', 'laughter', 'laughing', 'murmuring', 'chatter',
                       'aired', 'filmed', 'ladies', 'gentlemen', 'welcome', 'stage', 'transcript', 'netflix',
                  'apollo', 'like', 'goodnight', 'mutter', 'noo', 'nuh', 'oof', 'maan', 'cause', 'okay', 
                   'hey', 'also', 'someone', 'somebody', 'everybody', 'also', 'part' , 'sometimes', 'maybe', 
                   'three', 'second', 'everything', 'minute', 'name', 'kind', 'point', 'yeah', 'hello', 'one', 
                   'two', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'whine', 'hnn', 'malla', 'letta', 
                   'namoo', 'getta', 'nama', 'mana', 'chk','manoo', 'hadda', 'ama', 'carlin',
                  'go', 'know', 'host', 'goodnight', 'get', 'gon', 'think', 'say', 'right', 'look',
                  'thing', 'make', 'know', 'want', 'going', 'would', 'could', 'gentlemen', 'let', 'please',
                   'hbo', 'special' 'yes', 'take', 'say', 'got', 'come', 'see', 'really', 'tell',
                   'well', 'give', 'said']
                    
                    # 'know'? 'go'? 'fuck'?
    
                    # haven't left 'i'm' etc. as those should be cleaned up
                    # by a mixture of stopwords, punctuation removeal, lemmatizing and minimum length

data_lemmatized = data_lemmatized.apply(remove_stopw, args=(stopwords.words('english'),))
data_lemmatized = data_lemmatized.apply(remove_stopw, args=(words_to_remove,))

AttributeError: 'list' object has no attribute 'apply'

In [39]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 9), (1, 1), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 1), (8, 6), (9, 1), (10, 3), (11, 1), (12, 10), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 6), (19, 2), (20, 1), (21, 1), (22, 2), (23, 2), (24, 14), (25, 1), (26, 2), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 8), (35, 1), (36, 2), (37, 1), (38, 3), (39, 2), (40, 1), (41, 8), (42, 1), (43, 5), (44, 3), (45, 1), (46, 8), (47, 1), (48, 1), (49, 2), (50, 3), (51, 5), (52, 1), (53, 4), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 7), (66, 1), (67, 2), (68, 4), (69, 1), (70, 2), (71, 1), (72, 7), (73, 13), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 2), (82, 7), (83, 1), (84, 1), (85, 1), (86, 1), (87, 5), (88, 7), (89, 2), (90, 1), (91, 3), (92, 1), (93, 1), (94, 7), (95, 1), (96, 1), (97, 2), (98, 1), (99, 2), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 3), (107, 8), (108, 1), (109, 1), (110,

In [40]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [41]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(19,
  '0.000*"tia" + 0.000*"trickortreat" + 0.000*"quemado" + 0.000*"relleno" + '
  '0.000*"salio" + 0.000*"selfmade" + 0.000*"sexually_transmitte" + '
  '0.000*"smarta" + 0.000*"puta" + 0.000*"aborigine"'),
 (21,
  '0.000*"tia" + 0.000*"trickortreat" + 0.000*"quemado" + 0.000*"relleno" + '
  '0.000*"salio" + 0.000*"selfmade" + 0.000*"sexually_transmitte" + '
  '0.000*"smarta" + 0.000*"puta" + 0.000*"aborigine"'),
 (2,
  '0.000*"tia" + 0.000*"trickortreat" + 0.000*"quemado" + 0.000*"relleno" + '
  '0.000*"salio" + 0.000*"selfmade" + 0.000*"sexually_transmitte" + '
  '0.000*"smarta" + 0.000*"puta" + 0.000*"aborigine"'),
 (23,
  '0.000*"tia" + 0.000*"trickortreat" + 0.000*"quemado" + 0.000*"relleno" + '
  '0.000*"salio" + 0.000*"selfmade" + 0.000*"sexually_transmitte" + '
  '0.000*"smarta" + 0.000*"puta" + 0.000*"aborigine"'),
 (16,
  '0.000*"tia" + 0.000*"trickortreat" + 0.000*"quemado" + 0.000*"relleno" + '
  '0.000*"salio" + 0.000*"selfmade" + 0.000*"sexually_transmitte" + '
  '0.00

In [42]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.809475181638208

Coherence Score:  0.4098598915820103


In [43]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.254652 -0.465863       1        1  69.396247
26    -0.128585 -0.528875       2        1  16.496249
18    -0.446451 -0.297300       3        1   7.293023
4      0.174833 -0.478874       4        1   3.878451
0      0.397900 -0.306968       5        1   2.353887
12     0.305429  0.064093       6        1   0.418700
28     0.090416  0.177686       7        1   0.102399
17     0.012996  0.037871       8        1   0.028625
7     -0.042907  0.082220       9        1   0.023856
14    -0.014961  0.082836      10        1   0.004179
9     -0.004701  0.081659      11        1   0.000228
29    -0.004701  0.081659      12        1   0.000224
8     -0.004701  0.081659      13        1   0.000223
20    -0.004701  0.081659      14        1   0.000222
10    -0.004701  0.081659      15        1   0.000221
15    -0.004701  0.081659      16        1   0.000221
13    -0.004701  0.081659      17        1   0.000221
11    -0.004701  0.081659      18        1   0.000220
27    -0.004701  0.081659      19        1   0.000218
5     -0.004701  0.081659      20        1   0.000218
19    -0.004701  0.081659      21        1   0.000217
3     -0.004701  0.081659      22        1   0.000217
2     -0.004701  0.081659      23        1   0.000217
24    -0.004701  0.081659      24        1   0.000217
21    -0.004701  0.081659      25        1   0.000217
16    -0.004701  0.081659      26        1   0.000217
22    -0.004701  0.081659      27        1   0.000217
1     -0.004701  0.081659      28        1   0.000216
25    -0.004701  0.081659      29        1   0.000216
23    -0.004701  0.081659      30        1   0.000215, topic_info=           Term          Freq         Total Category  logprob  loglift
676         say  17503.000000  17503.000000  Default  30.0000  30.0000
702        shit  10424.000000  10424.000000  Default  29.0000  29.0000
368         guy  11322.000000  11322.000000  Default  28.0000  28.0000
346          go   6906.000000   6906.000000  Default  27.0000  27.0000
493         man  10381.000000  10381.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
25   always_ask      0.000055     35.597323  Topic30 -10.5279  -0.3402
26      amazing      0.000055   1061.093045  Topic30 -10.5279  -3.7349
27     american      0.000055    491.117899  Topic30 -10.5279  -2.9646
28      anymore      0.000055    857.297623  Topic30 -10.5279  -3.5217
29     anywhere      0.000055    271.118361  Topic30 -10.5279  -2.3704

[1745 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
0         1  0.999737              able
1         1  0.663747        absolutely
1         4  0.334686        absolutely
2         1  1.002590  absolutely_nothe
2365      2  0.990884        accomplish
...     ...       ...               ...
878       1  1.000163             write
880       1  0.956508              year
880       2  0.042870              year
880       3  0.000621              year
1440      2  0.999036              yell

[709 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 27, 19, 5, 1, 13, 29, 18, 8, 15, 10, 30, 9, 21, 11, 16, 14, 12, 28, 6, 20, 4, 3, 25, 22, 17, 23, 2, 26, 24])

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '../mallet/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=30, id2word=id2word)

Mallet LDA: 30 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 4254
total tokens: 800189
<10> LL/token: -10.67693
<20> LL/token: -10.06898
<30> LL/token: -9.81707
<40> LL/token: -9.68959

0	1.66667	people man time good lot thing show call happen feel job make place find person story live father understand comedy 
1	1.66667	shit man motherfucker bitch black talk woman goddamn boy real time back pussy good white put brother baby call crazy 
2	1.66667	country people child war time state care american planet reason year vote government live issue lot learn give bullshit freedom 
3	1.66667	girl dude man talk guy feel friend cool time weird face gay hate drunk happen shit pussy hard hit crazy 
4	1.66667	wife day back bed put people walk horse start kid side run life family house arm morning sleep hour lie 
5	1.66667	people car plane german drive call today back feel side love good police language fly suppose listen thing person chip 
6	1.66667	guy show great time play walk ba

<160> LL/token: -9.39171
<170> LL/token: -9.3886
<180> LL/token: -9.38383


In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
    

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=24, step=2)

In [ ]:
# Show graph
limit=24; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))


In [ ]:
# Select the model and print the topics
optimal_model = model_list[5]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics